In [ ]:
import numpy as np
import pandas as pd
dataset = pd.read_csv('train2.csv')
testset = pd.read_csv('test_without_labels2.csv')

In [2]:
dataset= dataset.fillna('')

In [3]:
for i in range(15):
    print(dataset.Question1[i])
    print(dataset.Question2[i])
    print()

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?

What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?

How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?

Why am I mentally very lonely? How can I solve it?
Find the remainder when [math]23^{24}[/math] is divided by 24,23?

Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Which fish would survive in salt water?

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?

Should I buy tiago?
What keeps childern active and far from phone and video games?

How can I be a good geologist?
What should I do to be a great geologist?

When do you use シ instea

In [4]:
def clean(text, stem_words=True):
    import re
    from string import punctuation
    
    if pd.isnull(text):
        return ''
    if type(text) != str or text=='':
        return ''

    # Clean the text
    text = re.sub("\'s", " ", text) 
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE) 
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE) 
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE) 
    text = re.sub("[c-fC-F]\:\/", " disk ", text) 
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text) #remove comma bn numbers
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    text = re.sub('[^\x00-\x7F]+', ' non_ascii_word ', text) 
    text = re.sub("(?<=[0-9])rs ", " rs ", text, flags=re.IGNORECASE) #rs is indian dollar
    text = re.sub(" rs(?=[0-9])", " rs ", text, flags=re.IGNORECASE)  
    
    # some more clean text rules suggested online
    text = re.sub(r" UK ", " England ", text, flags=re.IGNORECASE)
    text = re.sub(r" imrovement ", " improvement ", text, flags=re.IGNORECASE)
    text = re.sub(r" intially ", " initially ", text, flags=re.IGNORECASE)
    text = re.sub(r" dms ", " direct messages ", text, flags=re.IGNORECASE)  
    text = re.sub(r" demonitization ", " demonetization ", text, flags=re.IGNORECASE) 
    text = re.sub(r" actived ", " active ", text, flags=re.IGNORECASE)
    text = re.sub(r" kms ", " kilometers ", text, flags=re.IGNORECASE)
    text = re.sub(r" cs ", " computer science ", text, flags=re.IGNORECASE) 
    text = re.sub(r" upvote", " up vote", text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ", text, flags=re.IGNORECASE)
    text = re.sub(r" \0rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(r" calender ", " calendar ", text, flags=re.IGNORECASE)
    text = re.sub(r" ios ", " operating system ", text, flags=re.IGNORECASE)
    text = re.sub(r" gps ", " GPS ", text, flags=re.IGNORECASE)
    text = re.sub(r" gst ", " GST ", text, flags=re.IGNORECASE)
    text = re.sub(r" programing ", " programming ", text, flags=re.IGNORECASE)
    text = re.sub(r" bestfriend ", " best friend ", text, flags=re.IGNORECASE)
    text = re.sub(r" III ", " 3 ", text)
    text = re.sub(r" J K ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub('[0-9]+\.[0-9]+', " 87 ", text) #random number that is replaced below
    text = re.sub('[0-9]','number',text)
    text = ''.join([c for c in text if c not in punctuation]).lower()
    return text

In [5]:
df = pd.DataFrame()
df['question1'] = dataset['Question1'].apply(clean)
df['question2'] = dataset['Question2'].apply(clean)

In [6]:
#for test set
dft = pd.DataFrame()
dft['question1'] = testset['Question1'].apply(clean)
dft['question2'] = testset['Question2'].apply(clean)

In [7]:
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(token_pattern=r'[a-zA-Z_]{1,}', ngram_range=(2,3), max_features=5000)
vectorizer.fit(pd.concat((df['question1'],df['question2'])))

xt1 = vectorizer.transform(df['question1'].values)
xt2 = vectorizer.transform(df['question2'].values)
df1 = scipy.sparse.hstack((xt1,xt2))

xtt1 = vectorizer.transform(dft['question1'].values)
xtt2 = vectorizer.transform(dft['question2'].values)
dftest = scipy.sparse.hstack((xtt1,xtt2))

In [8]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report as clrp
kf = KFold(n_splits=5)
from xgboost import XGBClassifier
classifier2 = XGBClassifier(max_depth=50, n_estimators=80, colsample_bytree=0.7, alpha=4, objective='binary:logistic', verbose=1, subsample=0.8)

In [59]:
#from sklearn.decomposition import TruncatedSVD
#svd = TruncatedSVD(n_components = 100,random_state=42)
#svd.fit(scipy.sparse.vstack((df1,dftest)))
#trainx = svd.transform(df1)
#testx = svd.transform(dftest)

In [64]:
#classifier2.fit(df1,dataset.IsDuplicate)
df2=df1.todense()#for use in cross validation
df2.shape

(283004, 600)

In [9]:
tt = np.array(dataset.IsDuplicate)
tt.shape

(283004,)

In [77]:
for train_index, test_index in kf.split(df2,tt):
    classifier2.fit(df2[train_index],tt[train_index])
    ypred=classifier2.predict(df2[test_index])
    ytestt=tt[test_index]
    print(clrp(ytestt,ypred))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86     35456
           1       0.81      0.63      0.71     21145

    accuracy                           0.81     56601
   macro avg       0.81      0.77      0.78     56601
weighted avg       0.81      0.81      0.80     56601

              precision    recall  f1-score   support

           0       0.81      0.91      0.86     35675
           1       0.80      0.64      0.71     20926

    accuracy                           0.81     56601
   macro avg       0.81      0.77      0.79     56601
weighted avg       0.81      0.81      0.80     56601

              precision    recall  f1-score   support

           0       0.81      0.91      0.86     35431
           1       0.81      0.64      0.71     21170

    accuracy                           0.81     56601
   macro avg       0.81      0.77      0.78     56601
weighted avg       0.81      0.81      0.80     56601

              preci

In [10]:
classifier2.fit(df1,tt)
ypred = classifier2.predict(dftest)
naive = pd.DataFrame()

In [11]:
naive['Id'] = testset['Id']
naive['Predicted'] = ypred

In [74]:
naive.to_csv('naive2.csv',index=False)